Manual Mixer calibration
------------------------

The Cluster RF Modules (QCM-RF and QRM-RF) have integrated IQ mixers that handle both upconversion and downconversion of RF signals.
These mixers use a local oscillator (LO) with a frequency of $\omega_{LO}$.
During upconversion, they combine signals from the I and Q paths, which have a frequency of $\omega_{NCO}$, resulting in an output signal at $\omega_{LO} + \omega_{NCO}$.
However, due to the inherent mathematical imperfections of IQ mixers, the output also includes an unwanted signal at $\omega_{LO}$ (known as leakage) and another at $\omega_{LO} - \omega_{NCO}$ (called the undesired sideband).
Cluster baseband modules (QCM and QRM) can be used with external mixers, which have similar imperfections.


In this tutorial, we are going to look at ways to suppress the LO leakage and undesired sideband. This process is called mixer calibration.

To run this tutorial optimally, you will need:

* Spectrum analyzer

* SMA-cables

* Installation and enabling of ipywidgets:

```bash
        pip install ipywidgets
        jupyter nbextension enable --py widgetsnbextension
```

Setup
-----

First, we are going to import the required packages.

In [20]:
from __future__ import annotations

import json
from typing import TYPE_CHECKING, Callable

import ipywidgets as widgets
from ipywidgets import interact
from qcodes.instrument import find_or_create_instrument

from qblox_instruments import Cluster, ClusterType

if TYPE_CHECKING:
    from qblox_instruments.qcodes_drivers.module import Module

### Scan For Clusters

We scan for the available devices connected via ethernet using the Plug & Play functionality of the Qblox Instruments package (see [Plug & Play](https://docs.qblox.com/en/main/api_reference/tools.html#api-pnp) for more info).

`!qblox-pnp list`

In [2]:
cluster_ip = "192.168.0.20"
cluster_name = "qblox20"

### Connect to Cluster

We now make a connection with the Cluster.

In [3]:
cluster = find_or_create_instrument(
    Cluster,
    recreate=True,
    name=cluster_name,
    identifier=cluster_ip,
    # dummy_cfg=(
    #     {
    #         2: ClusterType.CLUSTER_QCM_RF,
    #         3: ClusterType.CLUSTER_QRM_RF,
    #         8: ClusterType.CLUSTER_QRM_RF,
    #         10: ClusterType.CLUSTER_QCM,
    #     }
    #     if cluster_ip is None
    #     else None
    # ),
)

#### Get connected modules

In [4]:
def get_connected_modules(cluster: Cluster, filter_fn: Callable | None = None) -> dict[int, Module]:
    def checked_filter_fn(mod: ClusterType) -> bool:
        if filter_fn is not None:
            return filter_fn(mod)
        return True

    return {
        mod.slot_idx: mod for mod in cluster.modules if mod.present() and checked_filter_fn(mod)
    }

In [5]:
# QCM-RF modules
modules_qrm = get_connected_modules(cluster, lambda mod: mod.is_qrm_type and mod.is_rf_type)
print('qrm:',modules_qrm)

# QCM-RF modules
modules_qcm = get_connected_modules(cluster, lambda mod: mod.is_qcm_type and mod.is_rf_type)
print('qcm:',modules_qcm)

qrm: {18: <Module: qblox20_module18 of Cluster: qblox20>, 20: <Module: qblox20_module20 of Cluster: qblox20>}
qcm: {8: <Module: qblox20_module8 of Cluster: qblox20>, 10: <Module: qblox20_module10 of Cluster: qblox20>, 12: <Module: qblox20_module12 of Cluster: qblox20>, 14: <Module: qblox20_module14 of Cluster: qblox20>}


In [21]:
# This uses the module of the correct type with the lowest slot index
module:Module = list(modules_qcm.values())[1]
print(module)

<Module: qblox20_module10 of Cluster: qblox20>


### Reset the Cluster

We reset the Cluster to enter a well-defined state. Note that resetting will clear all stored parameters, so resetting between experiments is usually not desirable.

In [7]:
cluster.reset()
print(cluster.get_system_status())

Status: OKAY, Flags: NONE, Slot flags: NONE


We upload a simple sequence program that keeps playing the DC waveform at 30 percent IF power (waveform amplitude = 0.3). This will be modulated and upconverted within the QRM-RF before outputting.

In [8]:
# Sequence program.
seq_prog = """
      wait_sync 4

loop: play    0,0,1200
      jmp     @loop
"""
waveforms = {"dc": {"data": [0.3 for i in range(0, 1200)], "index": 0}}

# Add sequence to single dictionary and write to JSON file.
sequence = {
    "waveforms": waveforms,
    "weights": {},
    "acquisitions": {},
    "program": seq_prog,
}
with open("sequence.json", "w", encoding="utf-8") as file:
    json.dump(sequence, file, indent=4)
    file.close()

module.sequencer0.sequence("sequence.json")

Let's configure the sequencer to generate an IF frequency of $100$ MHz. To get an output frequency of $5.0$ GHz, we then have to configure the LO to run at $4.9$ GHz.

In [22]:
lo_freq = 3260000000

if is_qrm_type:
    module.out0_in0_lo_freq(lo_freq) # For QRM_RF
else:
    module.out0_lo_freq(lo_freq)
    

In [28]:
f01 = 3102356879
nco_freq = f01-lo_freq
module.sequencer0.marker_ovr_en(True)
module.sequencer0.marker_ovr_value(3)  # Enables output on QRM-RF

# Configure the sequencer
module.sequencer0.mod_en_awg(True)
module.sequencer0.nco_freq(nco_freq)
module.sequencer0.sync_en(True)

module.arm_sequencer(0)
module.start_sequencer(0)

print(module.get_sequencer_status(0))

Status: OKAY, State: RUNNING, Info Flags: NONE, Warning Flags: NONE, Error Flags: NONE, Log: []


Connect the output of the QRM-RF (O1) to the spectrum analyzer. This is what the output looks like on the spectrum analyzer (center frequency at 4.85 GHz with 600 MHz bandwidth). We see three peaks which correspond to (from the right) 5 GHz (desired signal), 4.9 GHz (LO Leakage) and 4.8 GHz (unwanted sideband).

![IQ_Mixer_Calib_before.png](figures/IQ_Mixer_Calib_before.png)

To get better spurious free dynamic range (SFDR) and better suppression, we can manually calibrate the mixer by:

* Using DC offsets we can lower the LO leakage.
* By changing the gain ratio and phase of the IF signal we can cancel the unwanted sideband.

Create control sliders for thes parameters. Each time the value of a parameter is updated, the sequencer is automatically stopped from the embedded firmware for safety reasons and has to be manually restarted. The sliders cover the valid parameter range. If the code below is modified to input invalid values, the Cluster QRM-RF firmware will not program the values.

Execute the code below, move the sliders and observe the result on the spectrum analyzer.

In [29]:
def set_offset0(offset0: float) -> None:
    module.out0_offset_path0(offset0)

In [30]:
def set_offset1(offset1: float) -> None:
    module.out0_offset_path1(offset1)

In [33]:
def set_gain_ratio(gain_ratio: float) -> None:
    module.sequencer0.mixer_corr_gain_ratio(gain_ratio)
    # Start
    module.arm_sequencer(0)
    module.start_sequencer(0)


def set_phase_offset(phase_offset: float) -> None:
    module.sequencer0.mixer_corr_phase_offset_degree(phase_offset)
    # Start
    module.arm_sequencer(0)
    module.start_sequencer(0)


interact(
    set_offset0,
    offset0=widgets.FloatSlider(
        min=-14.0,
        max=14.0,
        step=0.001,
        value=-4.38,
        layout=widgets.Layout(width="1200px"),
    ),
)
interact(
    set_offset1,
    offset1=widgets.FloatSlider(
        min=-14.0,
        max=14.0,
        step=0.001,
        value=-5.20,
        layout=widgets.Layout(width="1200px"),
    ),
)
interact(
    set_gain_ratio,
    gain_ratio=widgets.FloatSlider(
        min=0.7, max=1.3, step=0.001, start=1.0, layout=widgets.Layout(width="1200px")
    ),
)
interact(
    set_phase_offset,
    phase_offset=widgets.FloatSlider(
        min=-45.0,
        max=45.0,
        step=0.001,
        start=0.0,
        layout=widgets.Layout(width="1200px"),
    ),
)

interactive(children=(FloatSlider(value=-4.38, description='offset0', layout=Layout(width='1200px'), max=14.0,…

interactive(children=(FloatSlider(value=-5.2, description='offset1', layout=Layout(width='1200px'), max=14.0, …

interactive(children=(FloatSlider(value=0.7, description='gain_ratio', layout=Layout(width='1200px'), max=1.3,…

interactive(children=(FloatSlider(value=0.0, description='phase_offset', layout=Layout(width='1200px'), max=45…

<function __main__.set_phase_offset(phase_offset: 'float') -> 'None'>

![IQ_Mixer_Calib_after.png](figures/IQ_Mixer_Calib_after.png)

Stop
----

Finally, let's stop the sequencers if they haven't already and close the instrument connection.

In [34]:
# Stop sequencer.
module.stop_sequencer()

# Print status of sequencer.
print(module.get_sequencer_status(0))

Status: OKAY, State: STOPPED, Info Flags: FORCED_STOP, Warning Flags: NONE, Error Flags: NONE, Log: []


Stop
----

Finally, let's stop the sequencers if they haven't already and close the instrument connection. One can also display a detailed snapshot containing the instrument parameters before
closing the connection by uncommenting the corresponding lines.

In [35]:
# Stop both sequencers.
module.stop_sequencer()

# Print status of both sequencers (should now say it is stopped).
print(module.get_sequencer_status(0))
print(module.get_sequencer_status(1))
print()

# Print an overview of the instrument parameters.
print("Snapshot:")
module.print_readable_snapshot(update=True)

# Reset the cluster
cluster.reset()
print(cluster.get_system_status())

Status: OKAY, State: STOPPED, Info Flags: FORCED_STOP, Warning Flags: NONE, Error Flags: NONE, Log: []
Status: OKAY, State: STOPPED, Info Flags: NONE, Warning Flags: NONE, Error Flags: NONE, Log: []

Snapshot:
qblox20_module10:
	parameter        value
--------------------------------------------------------------------------------
marker0_inv_en    :	False 
marker1_inv_en    :	False 
out0_att          :	0 (dB)
out0_lo_en        :	True 
out0_lo_freq      :	3260000000 (Hz)
out0_offset_path0 :	-4.407 (mV)
out0_offset_path1 :	-5.255 (mV)
out1_att          :	0 (dB)
out1_lo_en        :	True 
out1_lo_freq      :	6000000000 (Hz)
out1_offset_path0 :	7.625 (mV)
out1_offset_path1 :	7.625 (mV)
present           :	True 
qblox20_module10_sequencer0:
	parameter                       value
--------------------------------------------------------------------------------
connect_out0                     :	IQ 
connect_out1                     :	IQ 
cont_mode_en_awg_path0           :	False 
cont_mode_en_a